In [7]:
import numpy as np

# Define the RNN class
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize the dimensions
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.Wxh = np.random.randn(hidden_size, input_size)  # Input to hidden weights
        self.Whh = np.random.randn(hidden_size, hidden_size) # Hidden to hidden weights
        self.Why = np.random.randn(output_size, hidden_size) # Hidden to output weights
        self.bh = np.zeros((hidden_size, 1))                 # Hidden bias
        self.by = np.zeros((output_size, 1))                 # Output bias

    def forward(self, inputs):
        # Initialize hidden state
        h = np.zeros((self.hidden_size, 1))
        
        # Lists to store inputs, hidden states, and outputs for each time step
        xs, hs, ys, ps = [], [], [], []
        
        # Loop through each time step
        for t in range(len(inputs)):
            # Encode the input into a one-hot vector
            x = np.zeros((self.input_size, 1))
            x[inputs[t]] = 1
            
            # Update hidden state
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            
            # Compute output
            y = np.dot(self.Why, h) + self.by
            p = np.exp(y) / np.sum(np.exp(y))
            
            # Store input, hidden state, output, and prediction
            xs.append(x)
            hs.append(h)
            ys.append(y)
            ps.append(p)
        
        return xs, hs, ys, ps

# Example usage
if __name__ == "__main__":
    input_size = 5  # Size of input vector
    hidden_size = 3  # Size of hidden state
    output_size = 2  # Size of output vector
    sequence_length = 4  # Length of input sequence
    
    rnn = SimpleRNN(input_size, hidden_size, output_size)
    
    # Generate a sample input sequence (random integers)
    input_sequence = np.random.randint(input_size, size=sequence_length)
    
    # Forward pass through the RNN
    xs, hs, ys, ps = rnn.forward(input_sequence)
    
    # Print the input, hidden states, outputs, and predictions for each time step
    for t in range(sequence_length):
        print(f"Time step {t + 1} - Input: {input_sequence[t]}, Hidden state: {hs[t]}, Output: {ys[t]}, Prediction: {ps[t]}")


Time step 1 - Input: 3, Hidden state: [[-0.18080038]
 [-0.13572368]
 [-0.4667693 ]], Output: [[0.01215855]
 [0.98507456]], Prediction: [[0.27429966]
 [0.72570034]]
Time step 2 - Input: 0, Hidden state: [[-0.35705533]
 [ 0.69763657]
 [ 0.99345712]], Output: [[ 0.80030558]
 [-1.31206751]], Prediction: [[0.89209997]
 [0.10790003]]
Time step 3 - Input: 4, Hidden state: [[-0.53899675]
 [ 0.92199214]
 [-0.99283061]], Output: [[1.01655132]
 [1.04724745]], Prediction: [[0.49232657]
 [0.50767343]]
Time step 4 - Input: 4, Hidden state: [[-0.65878839]
 [-0.79178707]
 [-0.99551946]], Output: [[-0.14468846]
 [ 3.0924223 ]], Prediction: [[0.03779282]
 [0.96220718]]


We define a SimpleRNN class with an __init__ method to initialize the RNN's weights and biases and a forward method to perform the forward pass through the RNN.

The forward pass iterates through each time step, encoding the input at each time step into a one-hot vector, updating the hidden state using the RNN equation (with a tanh activation function), and computing the output and prediction.

The example usage section demonstrates how to create an instance of the SimpleRNN class and perform a forward pass with a randomly generated input sequence.

In [ ]:
import tensorflow as tf
import numpy as np

# Define the RNN model
class RNN(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()

        # Define the hidden layer
        self.hidden = tf.keras.layers.Dense(hidden_dim, activation='tanh')

        # Define the output layer
        self.output = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        # Pass the inputs through the hidden layer
        hidden = self.hidden(inputs)

        # Pass the hidden state through the output layer
        outputs = self.output(hidden)

        return outputs

# Load the real-time or time series dataset
dataset = 'dataset\ADANIPORTS.csv'

# Split the dataset into training and testing sets
train_set, test_set = ...

# Create the RNN model
model = RNN(input_dim=dataset.shape[1], hidden_dim=128, output_dim=1)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_set, train_set, epochs=100)

# Evaluate the model on the test set
test_loss = model.evaluate(test_set, test_set)

# Make a prediction on a new data point
new_data_point = ...

prediction = model.predict(new_data_point)


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the real-time or time series dataset
dataset_path = 'dataset/ADANIPORTS.csv'
data = pd.read_csv(dataset_path)

# Consider only relevant columns
data = data[['Close']].values  # Consider using a specific column like 'Close'

# Normalize the data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Split the dataset into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

# Function to create sequences for the RNN model
def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length:i+seq_length+1]
        sequences.append(seq)
        target.append(label)
    return np.array(sequences), np.array(target)

seq_length = 10  # Adjust as needed
train_sequences, train_target = create_sequences(train_data, seq_length)
test_sequences, test_target = create_sequences(test_data, seq_length)

# Create the RNN model
class RNN(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()

        self.hidden = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh')

        self.output = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        hidden = self.hidden(inputs)
        outputs = self.output(hidden)
        return outputs

model = RNN(input_dim=train_sequences.shape[-1], hidden_dim=128, output_dim=1)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_sequences, train_target, epochs=100, batch_size=32)

# Evaluate the model on the test set
test_loss = model.evaluate(test_sequences, test_target)

# Make a prediction on a new data point (assuming it's a sequence)
# Replace new_data_point with the appropriate input sequence
new_data_point = ...

prediction = model.predict(new_data_point)
